Q1

In [15]:
import numpy as np
import pandas as pd
import math
import scipy.stats as st
from scipy.optimize import minimize

# Set display options for comma separator and precision
pd.options.display.float_format = '{:,.2f}'.format

# Define the data
days = 365  # days_per_year
h = 0.15  # holding_cost_per_day
tc = 0.19  # transportation_cost_per_unit
ctc = 0.29  # centralized_transportation_cost
L = 5  # supply_leadtime_days
T = 6  # review_period_days
csl = 0.95  # critical service level
warehouse_cost = 50000  # annual warehouse cost

corr = 0

n_parts, n_regions = avg_demand.shape[0], avg_demand.shape[1]

# Raw data: average demand and standard deviation of demand for each part in each region
avg_demand = np.array([[35.48, 22.61, 17.66, 11.81, 3.36],
                       [2.48, 4.15, 6.15, 6.16, 7.49],
                       [0.48, 0.73, 0.80, 1.94, 2.54]])

std_demand = np.array([[6.98, 6.48, 5.26, 3.48, 4.49],
                       [3.16, 6.20, 6.39, 6.76, 3.56],
                       [1.98, 1.42, 2.39, 3.76, 3.98]])

num_demand = np.array([[10, 10, 10, 10, 10],
                       [20, 20, 20, 20, 20],
                       [70, 70, 70, 70, 70]])

# Calculate safety stock
def safety_stock_calculation(std_demand, service_level, lead_time, review_period):
    z_score = st.norm.ppf(service_level)
    safety_stock = z_score * std_demand * np.sqrt(lead_time + review_period)
    return safety_stock

safety_stock = safety_stock_calculation(std_demand, csl, L, T)

# Calculate total costs for decentralized (System A) and centralized (System B) systems
def total_cost(avg_demand, num_demand, safety_stock, h, tc, ctc, T, L):
    # Cycle stock for decentralized system
    cycle_stock_decentralized = avg_demand * T

    # Inventory cost for decentralized system
    inventory_cost_decentralized = np.sum(h * (cycle_stock_decentralized + safety_stock))

    # Transportation cost for decentralized system
    transport_cost_decentralized = tc * np.sum(num_demand * avg_demand)

    # Total cost for decentralized system
    total_cost_decentralized = inventory_cost_decentralized + transport_cost_decentralized

    # For the centralized system, we pool all demand and calculate cycle stock, safety stock, and transportation cost
    total_demand = np.sum(num_demand * avg_demand)
    total_safety_stock = np.sum(safety_stock)

    # Cycle stock for centralized system
    cycle_stock_centralized = total_demand * T

    # Inventory cost for centralized system
    inventory_cost_centralized = h * (cycle_stock_centralized + total_safety_stock)

    # Transportation cost for centralized system
    transport_cost_centralized = ctc * total_demand

    # Total cost for centralized system
    total_cost_centralized = inventory_cost_centralized + transport_cost_centralized

    return total_cost_decentralized, total_cost_centralized

# Calculate the total costs using the function
total_cost_decentralized, total_cost_centralized = total_cost(
    avg_demand, num_demand, safety_stock, h, tc, ctc, T, L
)

print(f"Total Cost for Decentralized System (A): ${total_cost_decentralized:,.2f}")
print(f"Total Cost for Centralized System (B): ${total_cost_centralized:,.2f}")


Total Cost for Decentralized System (A): $525.20
Total Cost for Centralized System (B): $2,305.84


Q2

In [12]:
import scipy.stats as st

# Re-define the safety stock calculation function
def safety_stock_calculation(avg_demand, std_demand, service_level, lead_time, review_period):
    z_score = st.norm.ppf(service_level)
    safety_stock = z_score * std_demand * np.sqrt(lead_time + review_period)
    return safety_stock

# Re-calculate the safety stock
safety_stock = safety_stock_calculation(avg_demand, std_demand, csl, L, T)

# Re-calculate total costs for decentralized (System A) and centralized (System B)
def total_cost(avg_demand, num_demand, safety_stock, h, tc, ctc, T, L):
    # Calculate cycle stock for decentralized system
    cycle_stock_decentralized = avg_demand * T

    # Calculate cycle stock for centralized system (sum of all individual cycle stocks)
    cycle_stock_centralized = np.sum(cycle_stock_decentralized)

    # Inventory cost for decentralized system (A)
    inventory_cost_decentralized = np.sum(h * (cycle_stock_decentralized + safety_stock))

    # Inventory cost for centralized system (B)
    inventory_cost_centralized = h * (cycle_stock_centralized + np.sum(safety_stock))

    # Transportation cost for decentralized system (A)
    transport_cost_decentralized = tc * np.sum(num_demand * avg_demand)

    # Transportation cost for centralized system (B)
    transport_cost_centralized = ctc * np.sum(num_demand * avg_demand)

    # Total cost for decentralized (A) and centralized (B) systems
    total_cost_decentralized = inventory_cost_decentralized + transport_cost_decentralized
    total_cost_centralized = inventory_cost_centralized + transport_cost_centralized

    return total_cost_decentralized, total_cost_centralized

# Calculate the total costs using the function
total_cost_decentralized, total_cost_centralized = total_cost(
    avg_demand, num_demand, safety_stock, h, tc, ctc, T, L
)

total_cost_decentralized, total_cost_centralized


(525.200395185698, 714.4103951856979)

In [13]:
### create correlation matrix using provided scalar value (corr can also be provided as raw data)
corr_matrix = np.full((n_regions, n_regions), corr)
np.fill_diagonal(corr_matrix, 1) # Set diagonal elements to 1

%clear

# Objective function to maximize (using matrix calculations)
def objective_function(flat_matrix):

    centralize = flat_matrix.reshape((3, 5))     # Reshape the flattened matrix to 3x5

    ### Compute aggregate standard deviation for centralized choices
    agg_std_demand = np.diag( np.dot( np.dot( np.multiply(std_demand, centralize), corr_matrix), np.transpose( np.multiply(std_demand, centralize) ) ) )
    sqrt_agg_std_demand = np.sqrt(agg_std_demand)

    sum_columns = np.sum(centralize, axis=1, keepdims=True)
    sum_columns[sum_columns == 0] = 1
    sum_columns_inv = 1 / sum_columns

    ### Adjust the standard deviation by dividing number of centralized regions for each part
    sqrt_agg_std_demand2 = np.repeat(sqrt_agg_std_demand.reshape(n_parts, 1), n_regions, axis=1)
    sum_columns_inv2 = np.repeat(sum_columns_inv.reshape(n_parts, 1), n_regions, axis=1)

    std_demand_updated = np.array( centralize * sum_columns_inv2 * sqrt_agg_std_demand2 + (1 - centralize) * std_demand )

    total_cost = sum(
        days * num_demand[i, j] *
        (h * ( st.norm.ppf(csl) * std_demand_updated[i, j] * math.sqrt(T + L) +  avg_demand[i, j] * (T/2 + L) )
        + ( avg_demand[i, j] * ( centralize[i, j] * ctc + (1 - centralize[i, j]) * tc ) )
        ) for i in range(n_parts) for j in range(n_regions) )

    return total_cost  # Negate for maximization



In [16]:
# Initial guess for decision variables
initial_guess = np.full((n_parts, n_regions), 0.5)
initial_guess_flat = initial_guess.flatten()     # Flatten the initial guess for use in optimization

# Bounds for decision variables (between 0 and 1)
bounds = [(0, 1)] * (n_parts*n_regions)

# Custom constraint enforcing the sum of each group (of 5) to not equal 1
def binary_constraints(x):
    # Binary constraint
    return [xi * (1 - xi) for xi in x]   # Forcing xi to be close to 0 or 1
    # return x - 0.5

# Constraints argument
constraints = ({'type': 'ineq', 'fun': binary_constraints})

# Optimization using minimize function from scipy
result = minimize(objective_function, initial_guess_flat, bounds=bounds, constraints=constraints)

# Extract the optimal solution as a 3x5 matrix
optimal_soln_mat = np.round((result.x).reshape((n_parts, n_regions)))

cdc_cost = days * sum( (num_demand[i, j] * avg_demand[i, j] * optimal_soln_mat[i, j]) for i in range(n_parts) for j in range(n_regions) )
investment = cdc_cost * 2 if cdc_cost <= 400000 else (800000 + (cdc_cost - 400000) * 1.5)

# Print the optimal solution as a 3x5 matrix
print("Correlation :", corr)
print(f"Minimum cost: ${(result.fun):>13,.2f}")
print(f"CDC cost    : ${cdc_cost:>13,.2f}")
print(f"Investment  : ${investment:>13,.2f}")
print("\nOptimal Solution:\n   R1   R2   R3   R4   R5")
for part in optimal_soln_mat:
    print(" ".join(f"{int(region):4}" for region in part))

Correlation : 0
Minimum cost: $ 1,272,712.13
CDC cost    : $   561,114.50
Investment  : $ 1,041,671.75

Optimal Solution:
   R1   R2   R3   R4   R5
   0    1    1    1    1
   1    1    1    1    1
   1    1    1    1    1
